In [2]:
from dotenv import load_dotenv
from pyflink.table.confluent import ConfluentSettings, ConfluentTools, ConfluentTableDescriptor
from pyflink.table import TableEnvironment, DataTypes, Schema
from pyflink.table.expressions import *
from pyflink.table import EnvironmentSettings, TableEnvironment
from pyflink.table.window import Tumble, Slide, Session
from pyflink.table.udf import udf

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import os
print(os.getenv("CLOUD_REGION"))

us-west1


In [5]:
settings = ConfluentSettings.from_global_variables()
settings = settings.in_streaming_mode()
env = TableEnvironment.create(settings)

It is important to have the same schema between Flink and Kafka Topic.

In [32]:
env.use_catalog("msds682")
env.use_database("msds682")
env.execute_sql("""DROP TABLE IF EXISTS `system.usage.day12`""")

In [51]:
env.execute_sql("""
    CREATE TABLE IF NOT EXISTS `system.usage.day12` (
        id STRING,
        cpu_usage FLOAT,
        cpu_stats ARRAY<FLOAT>,
        memory_usage FLOAT,
        `timestamp` TIMESTAMP(3),
        WATERMARK FOR `timestamp` AS `timestamp` - INTERVAL '5' SECOND
    ) WITH (
        'value.format'='avro-registry',
        'connector' = 'confluent'
    )
""")

In [52]:
table = env.from_path("`system.usage.day12`")

In [ ]:
windowed_table = table.window(Tumble.over(lit(4).second)
            .on(col("timestamp"))
            .alias("w")           
)
result = windowed_table.group_by(
    col("w"), 
    col("id")
).select(
    col("id"),
    col("memory_usage").min.alias("min_memory_usage"),
    col("memory_usage").max.alias("max_memory_usage"),
    col("memory_usage").count.alias("record_ct"),
    col("w").start.alias("window_start"),              
    col("w").end.alias("window_end")
)

In [ ]:
ConfluentTools.print_changelog(result)

| +I |                              1 |                           77.0 |                           77.0 |                    1 | 2025-11-25 05:29:36.000 | 2025-11-25 05:29:40.000 |
| +I |                              2 |                           77.3 |                           77.3 |                    1 | 2025-11-25 05:29:36.000 | 2025-11-25 05:29:40.000 |
| +I |                              3 |                           77.2 |                           77.2 |                    1 | 2025-11-25 05:29:36.000 | 2025-11-25 05:29:40.000 |
| +I |                              1 |                           77.1 |                           77.1 |                    1 | 2025-11-25 05:29:44.000 | 2025-11-25 05:29:48.000 |
| +I |                              3 |                           76.3 |                           76.3 |                    1 | 2025-11-25 05:30:48.000 | 2025-11-25 05:30:52.000 |
| +I |                              1 |                           77.0 |                       